In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [36]:
data = pd.read_csv("/Users/jmadu1/Documents/healthcare-dataset-stroke-data.csv")
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [37]:
#declare X and y
X = data.drop(columns='stroke')
y = data['stroke']

In [38]:
print(f'the sum of all duplicated data is {data.duplicated().sum()}. \n')

print("the sum of all null or NaN values is 201. see here: \n")
print(data.isnull().sum().sort_values(ascending=False))

the sum of all duplicated data is 0. 

the sum of all null or NaN values is 201. see here: 

bmi                  201
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
smoking_status         0
stroke                 0
dtype: int64


In [39]:
print("percentage of null or NaN values per feature: \n")
data.isnull().sum().sort_values(ascending=False)/len(data)

percentage of null or NaN values per feature: 



bmi                  0.039335
id                   0.000000
gender               0.000000
age                  0.000000
hypertension         0.000000
heart_disease        0.000000
ever_married         0.000000
work_type            0.000000
Residence_type       0.000000
avg_glucose_level    0.000000
smoking_status       0.000000
stroke               0.000000
dtype: float64

In [40]:
print("those values, turned into a dictionary: \n")
dict(data.isnull().sum().sort_values(ascending=False)/len(data))

those values, turned into a dictionary: 



{'bmi': 0.03933463796477495,
 'id': 0.0,
 'gender': 0.0,
 'age': 0.0,
 'hypertension': 0.0,
 'heart_disease': 0.0,
 'ever_married': 0.0,
 'work_type': 0.0,
 'Residence_type': 0.0,
 'avg_glucose_level': 0.0,
 'smoking_status': 0.0,
 'stroke': 0.0}

NaNs are only 3% of data, this is fine. bmi being NaN means data was not retrieved for random patient listed.

In [41]:
data.rename(columns={"hypertension": "ATP2B1"})

,id,gender,age,ATP2B1,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

In [79]:
from sklearn.dummy import DummyRegressor #Regressor that makes predictions using simple rules.

baseline_model = DummyRegressor(strategy="mean") # Baseline
baseline_model.fit(X_train, y_train) # Calculate value for stratgy
baseline_model.score(X_test, y_test) # Score model based on consistently predicting the strategy

-0.005619916856189899

In [80]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(baseline_model, X, y, cv=5)

In [81]:
pd.DataFrame(cv_results)

,fit_time,score_time,test_score
0,0.002389,0.000971,-0.322122
1,0.001114,0.000561,0.000000
2,0.000998,0.000388,0.000000
3,0.001460,0.000549,0.000000
4,0.001508,0.000514,0.000000


ordinal encoder - assigns number to each category

OHE - create binary column for each possible category

label encoder - encodes targets (this is reversable)

In [82]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

num_transformer = make_pipeline(SimpleImputer(), RobustScaler())
cat_transformer_1 = OneHotEncoder()
cat_transformer_2 = OrdinalEncoder()

preproc = make_column_transformer(
    (num_transformer, ['age', 'bmi','avg_glucose_level']),
    (cat_transformer_1, ['gender', 'ever_married']),
    (cat_transformer_2, ['work_type','Residence_type','smoking_status']),
    remainder='passthrough'
)

In [83]:
preproc

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 ['age', 'bmi', 'avg_glucose_level']),
                                ('onehotencoder', OneHotEncoder(),
                                 ['gender', 'ever_married']),
                                ('ordinalencoder', OrdinalEncoder(),
                                 ['work_type', 'Residence_type',
                                  'smoking_status'])])

In [84]:
# Train Pipeline
pipeline.fit(X_train,y_train)

# Make predictions
pipeline.predict(X_test.iloc[0:1])

# Score model
pipeline.score(X_train,y_train)

0.08227192078074796

In [85]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='l2', 
                           solver='lbfgs', 
                           max_iter=1000, 
                           verbose=1, 
                           n_jobs=None)

In [86]:
logreg_cv = cross_validate(model,
                           X, y,
                           cv=5,
                           scoring=['r2'])

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jmadu1/.pyenv/versions/3.10.6/envs/lewagon_current/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jmadu1/.pyenv/versions/3.10.6/envs/lewagon_current/lib/python3.10/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/jmadu1/.pyenv/versions/3.10.6/envs/lewagon_current/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1207, in fit
    X, y = self._validate_data(
  File "/Users/jmadu1/.pyenv/versions/3.10.6/envs/lewagon_current/lib